In [1]:
# If you are using different method to initialize spark session, this block might be different.
import findspark
findspark.init()

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.mllib.stat import Statistics

import numpy as np
from math import sqrt

sc = pyspark.SparkContext(appName="homework_1")
sqlContext = SQLContext(sc)

In [2]:
raw = sc.textFile("household_power_consumption.txt").map(lambda x: x.split(";"))
title = raw.first()
data = raw.filter(lambda x: x != title ).filter( lambda x: x[2] != "?" and x[3] != "?" and x[4] != "?" and x[5] != "?" ).map(lambda col: ( float(col[2]), float(col[3]), float(col[4]), float(col[5])))

In [3]:
summary = Statistics.colStats(data)

**1. Output the minimum, maximum, and count of the columns: 'global active power', 'global reactive power', 'voltage', and 'global intensity'.**  

In [4]:
print('[' + title[2] + ' ' + title[3] + ' ' + title[4] + ' ' + title[5] + ']')
print(summary.min())
print(summary.max())

[Global_active_power Global_reactive_power Voltage Global_intensity]
[7.600e-02 0.000e+00 2.232e+02 2.000e-01]
[ 11.122   1.39  254.15   48.4  ]


**2. Output the mean and standard deviation of these columns.**  

In [5]:
print(summary.mean())
print(sqrt(summary.variance()[0]),sqrt(summary.variance()[1]),sqrt(summary.variance()[2]),sqrt(summary.variance()[3]),)

[1.09161504e+00 1.23714476e-01 2.40839858e+02 4.62775931e+00]
1.0572941610939892 0.11272197955071644 3.2399866790099945 4.444396259786179


**3. Perform min-max normalization on the columns to generate normalized output.**  

In [6]:
def minMax_normalize(record, min_value, max_value):
    return (record - min_value) / (max_value - min_value)

In [7]:
mins = summary.min()
maxs = summary.max()

In [8]:
stds = data.map(lambda row: [minMax_normalize(row[0],mins[0],maxs[0]), minMax_normalize(row[1],mins[1],maxs[1]), minMax_normalize(row[2],mins[2],maxs[2]), minMax_normalize(row[3],mins[3],maxs[3])])
# Save the result to text file
stds.coalesce(1).saveAsTextFile('result.txt')